In [ ]:
import warnings
#warnings.simplefilter("ignore",category=ImportWarning)
warnings.filterwarnings('ignore')

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages 
import matplotlib.ticker
from matplotlib.ticker import FormatStrFormatter

import numpy as np
np.seterr(all="ignore")
from numpy import mean
from numpy import std
from numpy import cov
np.random.seed(12345)

import math

from astropy import units
from astropy.io import ascii
from astromodels.functions.function import (
    Function1D,
    FunctionMeta,
    ModelAssertionViolation,
)

from astropy.io import fits
import astromodels.functions.numba_functions as nb_func
import astropy.units as astropy_units
from astropy.table import Table

from scipy.optimize import curve_fit
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import ttest_ind

from threeML import *
from threeML.io.package_data import get_path_of_data_file
from threeML.io.logging import silence_console_log

from past.utils import old_div

import band

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import bandn
class Bandnew(Function1D, metaclass=FunctionMeta):
    r"""
    description :

        Band model from Band et al., 1993, parametrized with the peak energy

    latex : $K \begin{cases} \left(\frac{x}{piv}\right)^{\alpha} \exp \left(-\frac{(2+\alpha) x}{x_{p}}\right) & x \leq (\alpha-\beta) \frac{x_{p}}{(\alpha+2)} \\ \left(\frac{x}{piv}\right)^{\beta} \exp (\beta-\alpha)\left[\frac{(\alpha-\beta) x_{p}}{piv(2+\alpha)}\right]^{\alpha-\beta} &x>(\alpha-\beta) \frac{x_{p}}{(\alpha+2)} \end{cases} $

    parameters :

        K :

            desc : Differential flux at the pivot energy
            initial value : 1e-4
            min : 1e-50
            is_normalization : True
            transformation : log10

        alpha :

            desc : low-energy photon index
            initial value : -1.0
            min : -1.5
            max : 3

        xp :

            desc : peak in the x * x * N (nuFnu if x is a energy)
            initial value : 500
            min : 10
            transformation : log10

        width :

            desc : Full width at half maxima of the spectrum (log scale)
            initial value : 0.6
            min : 0.5
            max : 7.0
            
        fac :

            desc : guess of beta value
            initial value : -2.5
            fix : yes


    """
    
    def _set_units(self, x_unit, y_unit):
        self.K.unit = y_unit
        self.xp.unit = x_unit
        self.alpha.unit = astropy_units.dimensionless_unscaled
        self.fac.unit = astropy_units.dimensionless_unscaled
        self.width.unit = astropy_units.dimensionless_unscaled

    def evaluate(self, x, K, alpha, xp, width,fac):
        x_ = x
        if alpha < fac:
            alpha = fac
            log.warning("Alpha is currently less than beta. Setting alpha = beta.")

        if isinstance(x, astropy_units.Quantity):
            x_ = x.value
            alpha_ = alpha.value
            width_ = width.value
            K_ = K.value
            xp_ = xp.value
            fac_ = fac.value
            unit_ = self.y_unit
        else:
            unit_ = 1.0
            alpha_, width_, K_, x_, xp_,fac_ = alpha, width, K, x, xp,fac
       
        photar = np.empty_like(x_)
        for i in range(len(x_)):
            photar[i]=bandn.bandn(x_[i], alpha_, width_, xp_, fac_, K_)

        return photar*unit_


In [ ]:
font = {'family' : 'serif','weight' : 'bold','size' : 10}
matplotlib.rc('font',**font)
matplotlib.rc('grid',linewidth=1)
matplotlib.rc('xtick.major',width=2)
matplotlib.rc('xtick.major',size=5)
matplotlib.rc('xtick.minor',width=2)
matplotlib.rc('xtick.minor',size=3)
matplotlib.rc('ytick.major',width=2)
matplotlib.rc('ytick.major',size=5)
matplotlib.rc('ytick.minor',width=2)
matplotlib.rc('ytick.minor',size=3)

threeML_config.plugins.ogip.fit_plot.model_cmap = "Set1"
threeML_config.plugins.ogip.fit_plot.n_colors = 5

In [ ]:
# GRB data
triggerName = 'bn220426285'
ra = 60.8542
dec =  -75.3783
time_stamps = 10

trig = 672648596.000
t90 = 6.0

In [ ]:
def eval_spec(model,spec):
    p1 = '/home/soumya/work.dir/GRB/width_studies/Fermi_GRB/{0}/spec/{0}'.format(triggerName)
    aa = '/home/soumya/work.dir/GRB/width_studies/Fermi_GRB/{0}/output/{0}_{1}'.format(triggerName,model)

    file1 = open('{0}_Flux.dat'.format(aa),'a')

    with PdfPages('{0}_cornerplots.pdf'.format(aa)) as pdf1,PdfPages('{0}_model.pdf'.format(aa)) as pdf2,PdfPages('{0}_spectrum.pdf'.format(aa)) as pdf3:
        for j in range(1,time_stamps+1): 
            
            if (model=='Band_width'): bnd.fac=beta_val[i]
            spec_model = Model(PointSource(triggerName, ra, dec, spectral_shape=spec))
            model_no_eac = clone_model(spec_model)

            p1 = '/home/soumya/work.dir/GRB/width_studies/Fermi_GRB/{0}/spec/{0}'.format(triggerName)
            peak  = str(j)
            nai_0 = OGIPLike(det_name[0],("{0}_{1}_srcspectra.pha".format(p1,det[0])+"{"+peak+"}"),
                             ("{0}_{1}_bkgspectra.bak".format(p1,det[0])+"{"+peak+"}"),
                            ("{0}_{1}_weightedrsp.rsp".format(p1,det[0])+"{"+peak+"}"),spectrum_number=1,)

            nai_1 = OGIPLike(det_name[1],("{0}_{1}_srcspectra.pha".format(p1,det[1])+"{"+peak+"}"),
                             ("{0}_{1}_bkgspectra.bak".format(p1,det[1])+"{"+peak+"}"),
                            ("{0}_{1}_weightedrsp.rsp".format(p1,det[1])+"{"+peak+"}"),spectrum_number=1,)

            nai_a = OGIPLike(det_name[3],("{0}_{1}_srcspectra.pha".format(p1,det[3])+"{"+peak+"}"),
                             ("{0}_{1}_bkgspectra.bak".format(p1,det[3])+"{"+peak+"}"),
                            ("{0}_{1}_weightedrsp.rsp".format(p1,det[3])+"{"+peak+"}"),spectrum_number=1,)

            nai_2 = OGIPLike(det_name[2],("{0}_{1}_srcspectra.pha".format(p1,det[2])+"{"+peak+"}"),
                             ("{0}_{1}_bkgspectra.bak".format(p1,det[2])+"{"+peak+"}"),
                            ("{0}_{1}_weightedrsp.rsp".format(p1,det[2])+"{"+peak+"}"),spectrum_number=1,)

            nai_0.set_active_measurements("8-30","40-900")
            nai_1.set_active_measurements("8-30","40-900")
            nai_2.set_active_measurements("8-30","40-900")
            nai_a.set_active_measurements("8-30","40-900")
            #bgo_1.set_active_measurements("250-40000")


            data_list = DataList(nai_0,nai_1,nai_2,nai_a)
            BAsampler = BayesianAnalysis(model_no_eac, data_list)
            BAsampler.set_sampler("dynesty_nested", share_spectrum=True)
            BAsampler.sampler.setup(n_live_points=1000)
            BAsampler.sample()
            BAsampler.results.corner_plot()
            pdf1.savefig()
            plt.close()

            #-----------------------------------------------------------

            fig = display_spectrum_model_counts(BAsampler, step=False, min_rate=rate, data_colors=DataCOl[0:len(det)]
            ,model_colors=ModCOl[0:len(det)],model_labels=det_name,show_background=False,source_only=True)
            ax = fig.get_axes()[0]
            ax.set_ylim(1e-5,5e2)

            BAsampler.results.write_to("{0}_peak_{1}.fits".format(aa,peak), overwrite=True)
            pdf3.savefig()
            plt.close()

            fig2 = plot_spectra(BAsampler.results,ene_min=8, ene_max=1e4, num_ene=200,use_components= True,
                                flux_unit='keV2/(cm2 s keV)',equal_tailed=True,fit_cmap="Set1",contour_cmap="Set1")
            ax1 = fig2.get_axes()[0]
            plt.xlabel('Energy (keV)',fontsize=10)
            plt.ylabel(r'$\rm \nu$ $\rm F_{\rm \nu}$ $\rm (keV$ $\rm cm^{-2}$ $\rm sec^{-1})$',fontsize=10)
            ax1.set_ylim(1,1e4)
            ax1.legend(numpoints=1,prop={'size':8},loc='best',labelspacing=0.7)
            plt.tick_params(direction="in")
            plt.tick_params(axis='both',which='minor',length=3,width=1,labelsize=8)
            plt.tick_params(axis='both',which='major',length=3,width=1,labelsize=8)
            plt.tight_layout()
            pdf2.savefig()
            #------------------------------------------------------------
            flx1 = BAsampler.results.get_flux(0.939319932 * u.keV, 9393.19932369 * u.keV)
            flx2 = BAsampler.results.get_flux(8 * u.keV, 40000 * u.keV)

            print('{0} {1} {2} {3} {4} {5} {6}'.format(j,str(flx1['flux']['bn220426285: total'])[:-14],
                                                       str(flx1['low bound']['bn220426285: total'])[:-14],
                                                       str(flx1['hi bound']['bn220426285: total'])[:-14],
                                                       str(flx2['flux']['bn220426285: total'])[:-14],
                                                       str(flx2['low bound']['bn220426285: total'])[:-14],
                                                       str(flx2['hi bound']['bn220426285: total'])[:-14]),file=file1)

            #--------------------------------------------------------------------------------------------

    file1.close()

In [ ]:
DataCOl = ['xkcd:grey','xkcd:orange','xkcd:blue','xkcd:purple']
ModCOl = ['xkcd:black','xkcd:orange','xkcd:blue','xkcd:purple']

det_name = ['NaI0','NaI1','NaI2','NaIa']
det = ['n0','n1','n2','na']
rate = [30, 30,30,30]

In [ ]:
BndOld = Band()
model = 'BndOld'
spec = BndOld

BndOld.alpha.prior = Uniform_prior(lower_bound=-1.5, upper_bound=1.0)
BndOld.beta.prior = Uniform_prior(lower_bound=-7.0,upper_bound=-2.1)
BndOld.xp.prior = Uniform_prior(lower_bound=10, upper_bound=2000)
BndOld.K.prior = Log_uniform_prior(lower_bound=1e-5, upper_bound=1e5)

In [ ]:
eval_spec(model,spec)

In [ ]:
aa = '/home/soumya/work.dir/GRB/width_studies/Fermi_GRB/{0}/output/{0}_{1}'.format(triggerName,model)
epiv = 100.0
er = np.logspace(0,5,num=501) #in keV

In [ ]:
beta_val =[]
beta_nerr = []
beta_perr = []
width = []
for i in range(1,time_stamps+1):
    t2 = Table.read("{0}_peak_{1}.fits".format(aa,i),format='fits',hdu=1)
    beta_val.append(t2['VALUE'][3])
    beta_nerr.append(t2['NEGATIVE_ERROR'][3])
    beta_perr.append(t2['POSITIVE_ERROR'][3])
    
    param = [t2['VALUE'][1],t2['VALUE'][2],t2['VALUE'][3],epiv]
    xx = band.band(param,er)
    width.append(xx[1])

In [ ]:
bnd = Bandnew()
model = 'Band_width'
spec = bnd

bnd.alpha.prior = Uniform_prior(lower_bound=-1.5, upper_bound=1.0)
bnd.width.prior = Uniform_prior(lower_bound=np.min(width)-0.1,upper_bound=np.max(width)+0.1)
bnd.xp.prior = Uniform_prior(lower_bound=10, upper_bound=2000)
bnd.K.prior = Log_uniform_prior(lower_bound=1e-5, upper_bound=1e5)

In [ ]:
eval_spec(model,spec)

# TABLE CREATIONS

In [ ]:
det = 'n2'

In [ ]:
tp = Table.read('/home/soumya/work.dir/GRB/width_studies/Fermi_GRB/{0}/current/{0}_{1}_srcspectra.pha'.format(triggerName,det),
                format='fits',hdu=1)

tm = (tp['TSTART']-trig)+tp['TELAPSE']/2.0
tm_err = tp['TELAPSE']/2.0

#tm = tp['col1']
#tm_nerr = tp['col1']-tp['col3']
#tm_perr = tp['col4']-tp['col1']

t = Table.read('{0}_Flux_v11.dat'.format(aa),format='ascii')
f = open('{0}_param.txt'.format(aa),'w')

print('time time_err alpha alpha_nerr alpha_perr width width_nerr width_perr ep ep_nerr ep_perr beta beta_nerr beta_perr AIC BIC DIC logZ PDIC flx_z flz_z_lw flx_z_hi flx flz_lw flx_hi',file=f)

for i in range(1,time_stamps+1):
    hdul = fits.open("{0}_peak_{1}.fits".format(aa,i))
    t2 = Table.read("{0}_peak_{1}.fits".format(aa,i),format='fits',hdu=1)

    print('{0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10} {11} {12} {13} {14} {15} {16} {17} {18} {19} {20} {21}'.format(
        tm[i],tm_err[i],
        t2['VALUE'][1],t2['NEGATIVE_ERROR'][1],t2['POSITIVE_ERROR'][1],
        t2['VALUE'][3],t2['NEGATIVE_ERROR'][3],t2['POSITIVE_ERROR'][3],
        t2['VALUE'][2],t2['NEGATIVE_ERROR'][2],t2['POSITIVE_ERROR'][2],
        beta_val[i],beta_nerr[i],beta_perr[i],
        hdul[1].header['MV0'],hdul[1].header['MV1'],hdul[1].header['MV2'],
        hdul[1].header['MV3'],hdul[1].header['MV4'],
        t['col2'][i-1],t['col3'][i-1],t['col4'][i-1],
        t['col5'][i-1],t['col6'][i-1],t['col7'][i-1]),file=f)
f.close()

# PLOTS CREATIONS

In [ ]:
font = {'size' : 10}
matplotlib.rc('font',**font)
matplotlib.rc('grid',linewidth=1)
matplotlib.rc('xtick.major',width=2)
matplotlib.rc('xtick.major',size=7)
matplotlib.rc('xtick.minor',width=2)
matplotlib.rc('xtick.minor',size=4)
matplotlib.rc('ytick.major',width=2)
matplotlib.rc('ytick.major',size=7)
matplotlib.rc('ytick.minor',width=2)
matplotlib.rc('ytick.minor',size=4)

In [ ]:
det = 'n2' #brightest detector for lc
p1 = '/home/soumya/work.dir/GRB/width_studies/Fermi_GRB/{0}/current/glg_tte_{1}_{0}_v00.fit'.format(triggerName,det)

In [ ]:
t1 = Table.read('{0}_param.txt'.format(aa),format='ascii')

a = t1['alpha']
a_err = [-1.0*t1['alpha_nerr'],t1['alpha_perr']]
wd = t1['width']
wd_err = [-1.0*t1['width_nerr'],t1['width_perr']]
e = t1['ep']
e_nerr = [-1.0*t1['ep_nerr'],t1['ep_perr']]

tm = t1['time']
tm_err = t1['time_err']

mean_wd = np.mean(wd)

In [ ]:
############ INITIALIZATIONS
binsize = 0.1
tmin = -50.0
tmax = t90+50.0

t_b = np.arange(tmin,tmax+0.01,binsize)
zero_pos = np.absolute(t_b-0).argmin()

counts = 0 
prebkg = zero_pos-int(20/binsize)
postbkg = len(t_b)-int(50/binsize)
bkgtime =  t_b[prebkg]-t_b[0]+t_b[len(t_b)-1]-t_b[postbkg]
tplot = (t_b[1:]+t_b[:-1])/2.0 

############################# FERMI

t_data = Table.read(p1,format='fits',hdu=2)
t_data['TIME'] = t_data['TIME']-trig
t = t_data[(t_data['PHA']>=3) & (t_data['PHA']<=124) & (t_data['TIME']>tmin) & (t_data['TIME']<tmax)]

a= np.histogram(t['TIME'],bins=t_b)
abkg_cnt = binsize*(np.sum(a[0][0:prebkg])+np.sum(a[0][postbkg:len(a[0])]))/bkgtime
counts = a[0]-abkg_cnt

In [ ]:
nticks = 6     
par = 'beta'
################################# PLOTS
fig1,ax1 = plt.subplots(figsize=(10,5))
ax1.grid(alpha=0.3)

ax1.step(tplot,counts,where='mid',label='NaI 2 (8-900 keV)',color='darkred',linewidth=2)
plt.xlim(-5,t90+5)
ax1.set_ylabel('Counts/s',fontsize=14,color='darkred')
plt.tick_params(axis='y',colors='darkred',size=10,labelsize=12)
#ax1.legend(fancybox=True, framealpha=0.5,loc=1,fontsize=10)
ax1.set_xlabel('Time since Fermi trigger (+672648596.0 s)',fontsize=14)
plt.tick_params(axis='x',size=10,labelsize=12)

ax = ax1.twinx()
ax.errorbar(tm,wd,wd_err,tm_err,ls='--',marker='s',c='skyblue',mec='blue',ecolor='blue')
ax.set_ylabel('{0}'.format(par),fontsize=14,color='blue')
plt.tick_params(axis='y',colors='blue',size=10,labelsize=12)
ax.axhline(mean_wd,ls ='--',color='grey',label='Mean {0}: {1:.3f}'.format(par,mean_wd))

ax.yaxis.set_major_locator(matplotlib.ticker.LinearLocator(nticks))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))

plt.xlim(-2,t90+3)

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax.get_legend_handles_labels()

ax.legend(lines + lines2, labels + labels2, fancybox=True, framealpha=0.5,loc=1,fontsize=10)

plt.tight_layout()
plt.savefig('{0}_{1}_evolution.pdf'.format(aa,par),format='pdf')
plt.savefig('{0}_{1}_evolution.png'.format(aa,par),format='png')
plt.show()

##################################################################################

In [ ]:
min_ind = np.absolute(t_b+6.0).argmin() ; max_ind = np.absolute(t_b-(t90+10.0)).argmin()

nticks = 6     
################################# PLOTS
fig2,ax1 = plt.subplots(figsize=(10,5))
ax1.grid(alpha=0.3)

hist(t['TIME'], bins=t_b[min_ind:max_ind], histtype='stepfilled',color='xkcd:dull red',alpha=0.2,density=True)
edges = stats.bayesian_blocks(tplot[min_ind:max_ind],counts[min_ind:max_ind],50,fitness='measures',p0=0.1)
c,edges,patches = hist(t['TIME'], bins=edges, ax=ax, color='darkred',histtype='step', density=True)
plt.axhline(y = (c[0]+c[-1])/2.0,color='red',linestyle='--')

ax1.set_ylabel('P(t)',fontsize=14,color='darkred')
plt.tick_params(axis='y',colors='darkred',size=10,labelsize=12)

#plt.axvline(x = edges[1],color='green',linestyle='--')
#plt.axvline(x = edges[-2],color='green',linestyle='--')

ax1.set_xlabel('Time since Fermi trigger (+672648596.0 s)',fontsize=14)
plt.tick_params(axis='x',size=10,labelsize=12)

ax = ax1.twinx()
ax.errorbar(tm,wd,wd_err,[tm_nerr,tm_perr],ls='--',marker='s',c='skyblue',mec='blue',ecolor='blue')
ax.set_ylabel(r'$\mathscr{W}$',fontsize=14,color='blue')
plt.tick_params(axis='y',colors='blue',size=10,labelsize=12)
ax.axhline(mean_wd,ls ='--',color='grey',label=r'Mean $\mathscr{W}$'+' : {0:.3f}'.format(mean_wd))

ax.yaxis.set_major_locator(matplotlib.ticker.LinearLocator(nticks))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))

plt.xlim(-2,t90+3)

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax.get_legend_handles_labels()

ax.legend(lines + lines2, labels + labels2, fancybox=True, framealpha=0.5,loc=1,fontsize=10)

plt.tight_layout()
plt.savefig('{0}_width_evolution.pdf'.format(aa),format='pdf')
plt.savefig('{0}_width_evolution.png'.format(aa),format='png')
plt.show()

